<a href="https://colab.research.google.com/github/lauraroncancio/Notebooks-clase/blob/main/taller_4_%7BCastrell%C3%B3n_Juan%7D_%7BRoncancio_Laura%7D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taller 4
Juan Castrellón - Laura Roncancio

# Importacion de librarias

In [ ]:
!pip install torch -q
!pip install keras-tuner -q
!pip install transformers -q
!pip install "tf-models-official==2.13.*" -q

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import shutil
import pandas as pd
import numpy as np
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras_tuner as kt

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files

# import tensorflow_hub as hub # Commenting out the import of tensorflow_hub
# import tensorflow_text # Commenting out the import of tensorflow_text

from google.colab import files
from google.colab import userdata

import tensorflow_hub as hub
import tensorflow_text

# Descargar el dataset

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

!kaggle datasets download -d sbhatti/financial-sentiment-analysis
!unzip "financial-sentiment-analysis.zip"

Dataset URL: https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis
License(s): CC0-1.0
financial-sentiment-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  financial-sentiment-analysis.zip
replace data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data.csv                


In [ ]:
# Devidir los datos en train y test
data = pd.read_csv('/content/data.csv', sep=',')
data

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


In [ ]:
data["Sentiment"].value_counts()

,count
Sentiment,
neutral,3130
positive,1852
negative,860


# Preprocesamiento

In [ ]:
label_encoder = LabelEncoder()
data['Sentiment'] = label_encoder.fit_transform(data['Sentiment'])

etiquetas_unicas = label_encoder.classes_
for valor_numerico, etiqueta_original in enumerate(etiquetas_unicas):
    print(f'Valor numérico: {valor_numerico}, Etiqueta original: {etiqueta_original}')

Valor numérico: 0, Etiqueta original: negative
Valor numérico: 1, Etiqueta original: neutral
Valor numérico: 2, Etiqueta original: positive


In [ ]:
# Divide los datos en entrenamiento y prueba
train, test = train_test_split(data, test_size=0.2, stratify=data['Sentiment'], random_state=42, shuffle = True)

# Ahora divide el conjunto de entrenamiento en entrenamiento y validación
train, val = train_test_split(train, test_size=0.2, stratify=train['Sentiment'], random_state=42, shuffle = True)

print("Tamaño de datos de entrenamiento:", train.shape)
print("Tamaño de datos de validación:", val.shape)
print("Tamaño de datos de prueba:", test.shape)

train


Tamaño de datos de entrenamiento: (3738, 2)
Tamaño de datos de validación: (935, 2)
Tamaño de datos de prueba: (1169, 2)


,Sentence,Sentiment
3097,"Digia will also set up two subsidiaries , Digi...",1
5048,$BBRY Sierra. Has a great cash balance and imp...,2
5727,"Britain's FTSE gains, Land Securities up after...",2
185,The Finnish company sold its UK operation - co...,1
4265,Russian Media Ventures ' minority shareholder ...,1
...,...,...
326,( I&H ) in a move to enhance growth .,2
2821,"In addition , a further 29 employees can be la...",0
4365,"The paper industry 's de-inking sludge , which...",1
1603,$JE LOOKS like we are bouncing. Would be nice...,2


In [ ]:
X_train, X_test, X_val= train['Sentence'], test['Sentence'], val['Sentence']
y_train, y_test, y_val= train['Sentiment'], test['Sentiment'], val['Sentiment']

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 3, use_safetensors=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a

In [ ]:
def preprocess(texts):
    enc = tokenizer(
        texts.to_list(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="tf",
        return_attention_mask=True
    )
    return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"]}

In [ ]:
X_train_tokenized = preprocess(X_train)
X_test_tokenized = preprocess(X_test)
X_val_tokenized = preprocess(X_val)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955779 (255.42 MB)
Trainable params: 66955779 (255.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
optimizer = tf.optimizers.Adam(
    learning_rate=2e-5
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
metrics = ['accuracy']

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [ ]:
model.fit(X_train_tokenized, y_train,
          validation_data = (X_val_tokenized, y_val),
          epochs=20,
          #callbacks= EarlyStopping(monitor='val_accuracy', patience=4)
)

Epoch 1/20
117/117 [==============================] - 38s 123ms/step - loss: 0.7742 - accuracy: 0.6541 - val_loss: 0.5106 - val_accuracy: 0.7797
Epoch 2/20
117/117 [==============================] - 9s 75ms/step - loss: 0.4259 - accuracy: 0.8063 - val_loss: 0.4425 - val_accuracy: 0.7925
Epoch 3/20
117/117 [==============================] - 8s 67ms/step - loss: 0.2831 - accuracy: 0.8652 - val_loss: 0.5037 - val_accuracy: 0.7882
Epoch 4/20
117/117 [==============================] - 8s 69ms/step - loss: 0.2176 - accuracy: 0.8882 - val_loss: 0.5746 - val_accuracy: 0.7636
Epoch 5/20
117/117 [==============================] - 7s 57ms/step - loss: 0.1773 - accuracy: 0.9016 - val_loss: 0.5892 - val_accuracy: 0.7743
Epoch 6/20
117/117 [==============================] - 7s 60ms/step - loss: 0.1628 - accuracy: 0.9080 - val_loss: 0.6576 - val_accuracy: 0.7765
Epoch 7/20
117/117 [==============================] - 7s 61ms/step - loss: 0.1537 - accuracy: 0.9080 - val_loss: 0.6805 - val_accuracy: 0.77

In [ ]:
y_pred = model.predict(X_train_tokenized).logits
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred))

117/117 [==============================] - 4s 19ms/step
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       550
           1       0.95      0.95      0.95      2003
           2       1.00      1.00      1.00      1185

    accuracy                           0.94      3738
   macro avg       0.92      0.92      0.92      3738
weighted avg       0.94      0.94      0.94      3738



In [ ]:
y_pred = model.predict(X_test_tokenized).logits
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))

37/37 [==============================] - 1s 19ms/step
              precision    recall  f1-score   support

           0       0.37      0.35      0.36       172
           1       0.78      0.83      0.80       626
           2       0.87      0.79      0.83       371

    accuracy                           0.75      1169
   macro avg       0.67      0.66      0.66      1169
weighted avg       0.75      0.75      0.75      1169



1.¿Qué es Distilbert? ¿Mejoran los resultados? ¿El entrenamiento es más rápido? Compare los resultados con el modelo base del laboratorio.

Distilbert es una versión más liviana del modelo BERT, se usa para modelos de lenguaje natural cuando la capacidad computacional es limitada. Esta versión fue realizada con una técnica de destilación del conocimiento, la cual permite mantener la precisión.

Frente al modelo del laboratorio, se tienen mejores resultados en este modelo, ya que se tienen mayor accuracy tanto para train como para test, sin embargo, los resultados de este modelo Distilbert presentan indicios de un sobreajuste ya que para train el accuracy es 0.94 y para test es 0.75.

Sí, el entrenamiento de este modelo Distilbert es más rápido ya que reporta tiempos menores a los 10 segundos desde la segunda época en adelante, mientras que para el entremiento del laboratorio para ninguna época se demora menos de 11 segundos.


# Aumentado con GPT 2

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
model_name = "gpt2"
model_gpt = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer_gpt = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
def generate_negative_sentences(negative_text):
    input_ids = tokenizer_gpt.encode(negative_text, return_tensors="pt")

    output = model_gpt.generate(
        input_ids,
        max_length = len(negative_text) + 100,
        num_return_sequences=1,
        temperature=0.7,         # Temperatura (más alto = más creativo)
        top_k=50,                # Top 50 de palabras probables
        top_p=0.9,               # Centrado a palabras de alta probabilidad
        repetition_penalty=1.2,  # Reducción de repetición de palabras
        do_sample=True,           # Habilita el randomness del texto generado
    )

    generated_text = tokenizer_gpt.decode(output[0], skip_special_tokens=True)

    return generated_text.split('\n')[-1]

In [ ]:
neg = train[train['Sentiment']==0]

In [ ]:
negatives = []

neg_examples = ""

for i in range(1500):
    negatives_shown = neg.sample(n=3, random_state=i)
    for sentence in negatives_shown['Sentence']:
        neg_examples += sentence + '\n'
    prompt = f"""Help me generate more examples an negative texts such as the ones below {neg_examples}"""
    neg_examples = ""

    text = generate_negative_sentences(prompt)
    negatives.append(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
negatives_df = pd.DataFrame(negatives, columns=['Sentence'])
negatives_df['Sentiment'] = 0

train = pd.concat([train, negatives_df], ignore_index=True)

In [ ]:
train = train.sample(frac=1, random_state=42)

In [ ]:
X_train, X_test, X_val= train['Sentence'], test['Sentence'], val['Sentence']
y_train, y_test, y_val= train['Sentiment'], test['Sentiment'], val['Sentiment']

In [ ]:
X_train_tokenized = preprocess(X_train)
X_test_tokenized = preprocess(X_test)
X_val_tokenized = preprocess(X_val)

# Reentrenamiento con el aumentado

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 3, use_safetensors=False)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.optimizers.Adam(
    learning_rate=2e-5
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
metrics = ['accuracy']

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [ ]:
model.fit(X_train_tokenized, y_train,
          validation_data = (X_val_tokenized, y_val),
          epochs=20,
          #callbacks= EarlyStopping(monitor='val_accuracy', patience=4)
)

Epoch 1/20
164/164 [==============================] - 40s 130ms/step - loss: 0.6281 - accuracy: 0.7258 - val_loss: 0.4973 - val_accuracy: 0.7872
Epoch 2/20
164/164 [==============================] - 15s 95ms/step - loss: 0.3226 - accuracy: 0.8570 - val_loss: 0.4758 - val_accuracy: 0.7786
Epoch 3/20
164/164 [==============================] - 13s 79ms/step - loss: 0.2091 - accuracy: 0.9038 - val_loss: 0.4882 - val_accuracy: 0.7882
Epoch 4/20
164/164 [==============================] - 12s 70ms/step - loss: 0.1550 - accuracy: 0.9202 - val_loss: 0.5377 - val_accuracy: 0.7690
Epoch 5/20
164/164 [==============================] - 10s 60ms/step - loss: 0.1221 - accuracy: 0.9336 - val_loss: 0.6607 - val_accuracy: 0.7658
Epoch 6/20
164/164 [==============================] - 10s 61ms/step - loss: 0.1145 - accuracy: 0.9370 - val_loss: 0.6690 - val_accuracy: 0.7604
Epoch 7/20
164/164 [==============================] - 9s 57ms/step - loss: 0.1137 - accuracy: 0.9366 - val_loss: 0.6834 - val_accuracy:

In [ ]:
y_train.value_counts()

,count
Sentiment,
0,2050
1,2003
2,1185


In [ ]:
y_pred = model.predict(X_train_tokenized).logits
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred))

164/164 [==============================] - 5s 20ms/step
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2050
           1       0.95      0.94      0.94      2003
           2       1.00      1.00      1.00      1185

    accuracy                           0.96      5238
   macro avg       0.96      0.96      0.96      5238
weighted avg       0.96      0.96      0.96      5238



In [ ]:
y_pred = model.predict(X_test_tokenized).logits
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))

37/37 [==============================] - 1s 19ms/step
              precision    recall  f1-score   support

           0       0.32      0.40      0.35       172
           1       0.79      0.80      0.80       626
           2       0.88      0.76      0.82       371

    accuracy                           0.73      1169
   macro avg       0.66      0.65      0.66      1169
weighted avg       0.75      0.73      0.74      1169



Los resultados de este modelo, son muy parecidos al modelo anterior frente al accuracy, se tiene 0.96 para train y 0.73 para test, lo cual sugiere que también se presenta sobreajuste. Adicional, se nota para la evaluación de test que especialmente la categoría 0 presenta menos aciertos que las otras 2 categorías.

# Bert

In [ ]:
# Cargar los modelos de TensorFlow Hub con trainable=True si deseas fine-tuning
bert_preprocess_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    name='preprocessing'
)

bert_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",
    trainable=False, # Solo como extractor de características
    name='BERT_encoder'
)

print(f'BERT model selected: {bert_model}')
print(f'Preprocess model selected: {bert_preprocess_model}')


BERT model selected: <tensorflow_hub.keras_layer.KerasLayer object at 0x7e1b78c94530>
Preprocess model selected: <tensorflow_hub.keras_layer.KerasLayer object at 0x7e1a1c543290>


In [ ]:
# Capas Bert
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_output = bert_preprocess_model(text_input)
outputs = bert_model(preprocessing_output)

# Capas de nuestra red adicional
#l = tf.keras.layers.Dropout(0.1, name="dropout")
l = tf.keras.layers.Dense(512, activation='sigmoid', name="capaOculta1")(outputs['pooled_output'])
l = tf.keras.layers.Dropout(0.2, name="dropout")(l)
y = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

# Usamos inputs y outputs para construir el modelo final
model = tf.keras.Model(inputs=[text_input], outputs = [y])

In [ ]:
metrics = ['accuracy']

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=metrics)


In [ ]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [ ]:
model.fit(X_train, y_train,
          validation_data = (X_val, y_val),
          epochs=20,
          #callbacks= EarlyStopping(monitor='val_accuracy', patience=4)
          )

Epoch 1/20
164/164 [==============================] - 11s 38ms/step - loss: 0.7754 - accuracy: 0.6520 - val_loss: 0.8560 - val_accuracy: 0.6118
Epoch 2/20
164/164 [==============================] - 6s 35ms/step - loss: 0.6375 - accuracy: 0.7106 - val_loss: 0.7670 - val_accuracy: 0.6321
Epoch 3/20
164/164 [==============================] - 6s 34ms/step - loss: 0.6164 - accuracy: 0.7243 - val_loss: 0.7464 - val_accuracy: 0.6695
Epoch 4/20
164/164 [==============================] - 6s 35ms/step - loss: 0.6071 - accuracy: 0.7297 - val_loss: 0.7621 - val_accuracy: 0.6289
Epoch 5/20
164/164 [==============================] - 6s 35ms/step - loss: 0.5757 - accuracy: 0.7449 - val_loss: 0.7740 - val_accuracy: 0.6182
Epoch 6/20
164/164 [==============================] - 6s 35ms/step - loss: 0.5623 - accuracy: 0.7468 - val_loss: 0.8353 - val_accuracy: 0.6203
Epoch 7/20
164/164 [==============================] - 6s 35ms/step - loss: 0.5558 - accuracy: 0.7524 - val_loss: 0.7146 - val_accuracy: 0.672

In [ ]:
y_pred = model.predict(X_train)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred))

164/164 [==============================] - 5s 30ms/step
              precision    recall  f1-score   support

           0       0.84      0.87      0.85      2050
           1       0.81      0.83      0.82      2003
           2       0.76      0.68      0.72      1185

    accuracy                           0.81      5238
   macro avg       0.80      0.79      0.80      5238
weighted avg       0.81      0.81      0.81      5238



In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))

37/37 [==============================] - 1s 27ms/step
              precision    recall  f1-score   support

           0       0.29      0.36      0.32       172
           1       0.76      0.78      0.77       626
           2       0.67      0.56      0.61       371

    accuracy                           0.65      1169
   macro avg       0.57      0.57      0.57      1169
weighted avg       0.66      0.65      0.65      1169



Para este modelo, se percibe una mejora en el accuracy, a pesar de tener diferencia entre train 0.81 y test 0.65, esta diferencia es menor que para los modelos anteriores. Sin embargo, la categoría 0 continúa siendo la menos acertada en el modelo, mientras que las categorías 1 y 2 disminuyeron en sus métricas de evaluación para test frente a la versión del modelo del punto 1.
Por lo anterior, se sugiere adicionar también busqueda de hiperparámetros para corregir el sobreajuste y mejorar la clasificación.